# Import

In [1]:
from dotenv import load_dotenv
import os
import json
from typing import List, Union

from src.core.trajectory import Trajectory
from src.core.messages import (
    BaseMessage,
    UserMessage,
    AIMessage,
    ToolCallMessage,
    ToolCallOutputMessage,
    ChatMessageList,
    ToolMessageList
)
from src.core.tools import TOOLS
from src.env.appworld_env import AppWorldEnv
from src.llm.openai_client import OpenAIClient
from src.agent.base import BaseAgent

from src.agent.react import ReActAgent

from openai import OpenAI

from appworld import AppWorld, load_task_ids

# Env Setup

In [2]:
env_path = os.path.join("config", ".env")

load_dotenv(env_path)

True

# OpenAI Response API

## Simple response request

In [ ]:
llm_client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

In [10]:
response = llm_client.responses.create(
    model="gpt-4o",
    temperature=0.0,
    input=[
        {'role' : 'system', 'content' : '당신의 이름은 ReflACE Agent 입니다.'},
        {'role' : 'user', 'content' : '안녕하세요 당신은 누구인가요?'}
    ]
)

# Output Parsing
if hasattr(response, "output"):
    for output in response.output:
        if hasattr(output, "content"):
            for content in output.content:
                if hasattr(content, "text"):
                    print(content.text)

안녕하세요! 저는 ReflACE Agent입니다. 다양한 질문에 답변하고 도움을 드릴 수 있는 AI입니다. 무엇을 도와드릴까요?


In [11]:
print(response)

Response(id='resp_000b7636b507e0e900694382c64b4c81939aa2cc96dedb10a1', created_at=1766032070.0, error=None, incomplete_details=None, instructions=None, metadata={}, model='gpt-4o-2024-08-06', object='response', output=[ResponseOutputMessage(id='msg_000b7636b507e0e900694382c700908193b54878f8b067e23c', content=[ResponseOutputText(annotations=[], text='안녕하세요! 저는 ReflACE Agent입니다. 다양한 질문에 답변하고 도움을 드릴 수 있는 AI입니다. 무엇을 도와드릴까요?', type='output_text', logprobs=[])], role='assistant', status='completed', type='message')], parallel_tool_calls=True, temperature=0.0, tool_choice='auto', tools=[], top_p=1.0, background=False, conversation=None, max_output_tokens=None, max_tool_calls=None, previous_response_id=None, prompt=None, prompt_cache_key=None, reasoning=Reasoning(effort=None, generate_summary=None, summary=None), safety_identifier=None, service_tier='default', status='completed', text=ResponseTextConfig(format=ResponseFormatText(type='text'), verbosity='medium'), top_logprobs=0, truncation='di

## Tool Call


| Field | Description |
| :---: | :---: |
| type	| This should always be function |
| name	| The function's name (e.g. get_weather) |
| description	| Details on when and how to use the function |
| parameters	| JSON schema defining the function's input arguments |
| strict	| Whether to enforce strict mode for the function call |

In [4]:
TOOLS = [
    {
        "type": "function",
        "name": "get_weather",
        "description": """
        이 도구는 특정 도시의 현재 날씨를 조회하기 위해 사용합니다.
        
        이 도구를 활용하기 위한 입력 인자는 다음과 같습니다.
        <입력 인자>
        location: 도시 이름
        units: 온도 단위 (celsius, fahrenheit, None)
        </입력 인자>

        이 도구를 사용하기 위한 JSON Schema 예시는 다음과 같습니다.
        <JSON Schema>
        {
            "location": "Seoul, KR",
            "units": "celsius"
        }
        </JSON Schema>
        """,
        "strict": True,
        "parameters": {
            "type": "object",
            "properties": {
                "location": {"type": "string", "description": "예: Seoul, KR"},
                "units": {"type": ["string", "null"], "enum": ["celsius", "fahrenheit", None]},
            },
            "required": ["location", "units"],
            "additionalProperties": False,
        },
    },
]

In [3]:
def get_weather(location, units=None):
    return f"Weather in {location} is 10 {units}."

In [25]:
from typing import Literal

def tmp(a : int, b:Literal["1", "2"]):
    return a, b

In [23]:
llm_client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

trajectory = [
    {'role' : 'system', 'content' : '당신의 이름은 ReflACE Agent 입니다.'},
    {'role' : 'user', 'content' : '현재 서울의 날씨는 어떤가요?'}
]

response = llm_client.responses.create(
    model='gpt-4o',
    temperature=0.0,
    tools=TOOLS,
    input=trajectory,
)

trajectory += response.output

for output in response.output:
    if output.type == "function_call":
        print("✅ Tool Call Maded!")
        if output.name == "get_weather":
            # execute tool function
            tool_result = get_weather(**json.loads(output.arguments))
            print("✅ Tool Function Executed!")
            print(f"     📍 Result : {tool_result}")

            # add tool result to trajectory
            trajectory.append({
                'type' : 'function_call_output',
                'call_id' : output.call_id,
                'output' : json.dumps({
                    "weather_info" : tool_result
                })
            })           

response = llm_client.responses.create(
    model='gpt-4o',
    temperature=0.0,
    tools=TOOLS,
    input=trajectory,
)

for output in response.output:
    if output.type == "message":
        for content in output.content:
            if content.type == "output_text":
                print(content.text)

✅ Tool Call Maded!
✅ Tool Function Executed!
     📍 Result : Weather in Seoul, KR is 10 celsius.
서울의 현재 날씨는 섭씨 10도입니다.


In [24]:
trajectory

[{'role': 'system', 'content': '당신의 이름은 ReflACE Agent 입니다.'},
 {'role': 'user', 'content': '현재 서울의 날씨는 어떤가요?'},
 ResponseFunctionToolCall(arguments='{"location":"Seoul, KR","units":"celsius"}', call_id='call_gtDiWI8sjnkqFxWcZddRODkT', name='get_weather', type='function_call', id='fc_02d448c8cd97ffb900694387d9003c81908123434c62c606c3', status='completed'),
 {'type': 'function_call_output',
  'call_id': 'call_gtDiWI8sjnkqFxWcZddRODkT',
  'output': '{"weather_info": "Weather in Seoul, KR is 10 celsius."}'}]

## AppWorld

In [ ]:
def iterate_agent(
    agent:BaseAgent, 
    env:AppWorldEnv, 
    max_iter:int=30
):
    trajectory = Trajectory([UserMessage(content=agent._build_prompt(env))])

    for _ in range(max_iter):
        trajectory = agent.run(env, trajectory)

        if isinstance(trajectory.messages[-1], AIMessage) or "complete_task" in trajectory.messages[-1].content:
            return {
                'finished' : True,
                'trajectory' : trajectory
            }
        else:
            return {
                'finished' : False,
                'trajectory' : trajectory
            }

In [4]:
env = AppWorldEnv(task_type="train")

env.set_env(experiment_name="sample")

with open(os.path.join('src','prompt','react','system_prompt.txt'), 'r', encoding='utf-8') as f:
    system_prompt = f.read()

llm_client = OpenAIClient(
    model_name='gpt-4o',
    temperature=0.6,
    tools=TOOLS,
    system_prompt=system_prompt
)

react_agent = ReActAgent(llm_client=llm_client)

agent_action_result = iterate_agent(
    agent=react_agent,
    env=env,
    max_iter=30
)

if agent_action_result['finished']:
    print("✅ Task Finished")
else:
    print("❌ Task Not Finished")

NameError: name 'self' is not defined

/opt/anaconda3/envs/reflace/lib/python3.11/site-packages/IPython/core/interactiveshell.py:3707: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
agent_action_result['trajectory']

Trajectory(messages=UserMessage(content=Using these APIs, now generate code to solve the actual task:

My name is: Joyce Weaver. My personal email is joyce-weav@gmail.com and phone number is 3155673041.
Task: What is the title of the most-liked song in my Spotify playlists.
)

ToolCallMessage(type=function_call, call_id=call_bOlewVwjk6hU2KUGoH0NOjmx, name=appworld_execute, arguments={"code":"print(apis.api_docs.show_app_descriptions())"})

ToolCallOutputMessage(type=function_call_output, call_id=call_bOlewVwjk6hU2KUGoH0NOjmx, output=[
 {
  "name": "api_docs",
  "description": "An app to search and explore API documentation."
 },
 {
  "name": "supervisor",
  "description": "An app to access supervisor's personal information, account credentials, addresses, payment cards, and manage the assigned task."
 },
 {
  "name": "amazon",
  "description": "An online shopping app to buy products and manage orders, returns, etc."
 },
 {
  "name": "phone",
  "description": "An app to find and manage 

In [5]:
evaluation = env.env.evaluate()

evaluation.report()

────────────────────────────────────────────────── Overall Stats ──────────────────────────────────────────────────

Num Passed Tests : 2

Num Failed Tests : 0

Num Total  Tests : 2

───────────────────────────────────────────────────── Passes ──────────────────────────────────────────────────────

...

>> Passed Requirement

assert no model changes.

────────────────────────────────────────────────────── Fails ──────────────────────────────────────────────────────

None

''